# LEAPS Discovery Screener

This notebook discovers LEAPS-oriented opportunities from a dynamic equity universe, applies a multi-factor score, and prepares a watchlist for trade-readiness deep dive.

**Outputs**

- ranked LEAPS watchlist
- constrained model portfolio with sector caps
- share package in `outputs/`


In [6]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

from notebook_pipeline import (
    setup_report_style,
    parse_env_list,
    display_table,
    screen_universe,
    fetch_underlying_metrics,
    fetch_option_candidates,
    score_option_candidates,
    build_correlation_matrix,
)

setup_report_style()

FIGURE_COUNTER = 0


def show_figure(fig, title):
    global FIGURE_COUNTER
    FIGURE_COUNTER += 1
    if not str(title).lower().startswith("figure"):
        title = f"Figure {FIGURE_COUNTER}. {title}"
    fig.update_layout(title=title)
    fig.show()


In [7]:
USE_SCREEN = os.getenv("USE_SCREEN", "1") == "1"
QUICK_RUN = os.getenv("QUICK_RUN", "0") == "1"
TICKER_OVERRIDE = parse_env_list("TICKER_OVERRIDE")

MAX_TICKERS = int(os.getenv("MAX_TICKERS", "45"))
RATE_LIMIT_SLEEP = float(os.getenv("RATE_LIMIT_SLEEP", "0.20"))


STARTING_BALANCE = float(os.getenv("STARTING_BALANCE", "15000"))
MAX_PER_SECTOR = int(os.getenv("MAX_PER_SECTOR", "3"))
PORTFOLIO_NAMES = int(os.getenv("PORTFOLIO_NAMES", "10"))

if QUICK_RUN:
    MAX_TICKERS = min(MAX_TICKERS, 12)
    RATE_LIMIT_SLEEP = min(RATE_LIMIT_SLEEP, 0.10)

HORIZONS = {
    "short": {"min_dte": 7, "max_dte": 35, "target_dte": 21},
    "medium": {"min_dte": 36, "max_dte": 140, "target_dte": 75},
    "leaps": {"min_dte": 260, "max_dte": 760, "target_dte": 420},
}

CALL_MONEYNESS = {
    "short": (0.95, 1.05),
    "medium": (0.90, 1.08),
    "leaps": (0.75, 1.05),
}

config_df = pd.DataFrame(
    {
        "Parameter": [
            "USE_SCREEN",
            "QUICK_RUN",
            "MAX_TICKERS",
            "PORTFOLIO_NAMES",
            "MAX_PER_SECTOR",
        ],
        "Value": [USE_SCREEN, QUICK_RUN, MAX_TICKERS, PORTFOLIO_NAMES, MAX_PER_SECTOR],
    }
)
display_table(config_df, caption="Discovery Configuration")


,Parameter,Value
0,USE_SCREEN,True
1,QUICK_RUN,False
2,MAX_TICKERS,45
3,PORTFOLIO_NAMES,10
4,MAX_PER_SECTOR,3


In [8]:
tickers = screen_universe(
    use_screen=USE_SCREEN,
    ticker_override=TICKER_OVERRIDE,
    max_tickers=MAX_TICKERS,
    size=max(80, MAX_TICKERS * 3),
)

underlying_df = fetch_underlying_metrics(
    tickers,
    history_period="1y",
    rate_limit_sleep=RATE_LIMIT_SLEEP,
)
if underlying_df.empty:
    raise RuntimeError("No underlyings available for discovery run.")

call_frames = []
for _, row in underlying_df.iterrows():
    call_df = fetch_option_candidates(
        row["ticker"],
        side="call",
        spot=float(row["spot"]),
        horizons=HORIZONS,
        moneyness_bounds=CALL_MONEYNESS,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
    )
    if not call_df.empty:
        call_frames.append(call_df)

chain_df = pd.concat(call_frames, ignore_index=True) if call_frames else pd.DataFrame()
if chain_df.empty:
    raise RuntimeError("No LEAPS call candidates found.")

scored_df = score_option_candidates(chain_df, underlying_df)
scored_df = scored_df[scored_df["side"] == "call"].copy()

scored_df["discovery_score"] = (
    0.55 * scored_df["master_score"]
    + 0.25 * scored_df["fundamental_score"]
    + 0.20 * scored_df["trend_score"]
)

watchlist_df = (
    scored_df.sort_values("discovery_score", ascending=False)
    .groupby("ticker", as_index=False)
    .first()
    .sort_values("discovery_score", ascending=False)
)

portfolio_rows = []
sector_counts = {}
for _, row in watchlist_df.iterrows():
    sector = row.get("sector", "Unknown")
    if sector_counts.get(sector, 0) >= MAX_PER_SECTOR:
        continue
    portfolio_rows.append(row)
    sector_counts[sector] = sector_counts.get(sector, 0) + 1
    if len(portfolio_rows) >= PORTFOLIO_NAMES:
        break

portfolio_df = pd.DataFrame(portfolio_rows)
if not portfolio_df.empty:
    total = portfolio_df["discovery_score"].sum()
    portfolio_df["weight"] = (
        portfolio_df["discovery_score"] / total if total > 0 else 1 / len(portfolio_df)
    )

display_table(
    watchlist_df[
        [
            "ticker",
            "sector",
            "horizon",
            "dte",
            "strike",
            "mid",
            "iv",
            "expected_return",
            "discovery_score",
        ]
    ].head(20),
    caption="Discovery Watchlist",
    format_dict={
        "strike": "${:,.2f}",
        "mid": "${:,.2f}",
        "iv": "{:.1%}",
        "expected_return": "{:.1%}",
        "discovery_score": "{:.1f}",
    },
)

if not portfolio_df.empty:
    display_table(
        portfolio_df[["ticker", "sector", "horizon", "discovery_score", "weight"]],
        caption="Model Portfolio",
        format_dict={"discovery_score": "{:.1f}", "weight": "{:.1%}"},
    )


,ticker,sector,horizon,dte,strike,mid,iv,expected_return,discovery_score
25,MU,Technology,medium,96,$400.00,$58.12,75.2%,427.9%,91.3
10,CMCSA,Communication Services,medium,68,$32.50,$1.01,27.8%,557.6%,82.0
43,VZ,Communication Services,medium,68,$49.00,$0.62,20.2%,1262.5%,81.4
13,F,Consumer Cyclical,leaps,341,$12.00,$0.81,0.0%,380.0%,81.0
15,GOOGL,Communication Services,medium,68,$320.00,$21.07,35.6%,166.7%,80.7
14,GOOG,Communication Services,medium,68,$325.00,$18.38,34.8%,179.9%,80.7
5,BAC,Financial Services,medium,68,$60.00,$1.15,25.3%,182.1%,78.7
41,T,Communication Services,medium,68,$29.00,$0.36,21.8%,564.2%,77.4
28,NU,Financial Services,short,26,$17.00,$1.06,54.5%,121.9%,76.7
32,PFE,Healthcare,leaps,341,$27.00,$2.60,24.3%,139.8%,75.0


,ticker,sector,horizon,discovery_score,weight
25,MU,Technology,medium,91.3,11.5%
10,CMCSA,Communication Services,medium,82.0,10.3%
43,VZ,Communication Services,medium,81.4,10.3%
13,F,Consumer Cyclical,leaps,81.0,10.2%
15,GOOGL,Communication Services,medium,80.7,10.2%
5,BAC,Financial Services,medium,78.7,9.9%
28,NU,Financial Services,short,76.7,9.7%
32,PFE,Healthcare,leaps,75.0,9.5%
0,AAL,Industrials,short,73.4,9.3%
3,APLD,Technology,leaps,72.5,9.1%


In [9]:
fig_top = px.bar(
    watchlist_df.head(15).sort_values("discovery_score"),
    x="discovery_score",
    y="ticker",
    color="horizon",
    orientation="h",
    hover_data=["sector", "dte", "expected_return", "iv"],
    height=560,
    color_discrete_sequence=["#1F3A5F", "#4C6E91", "#8B9BB4"],
)
show_figure(fig_top, "Discovery Score by Ticker")

sector_summary = (
    watchlist_df.groupby("sector", as_index=False)
    .agg(score=("discovery_score", "mean"), names=("ticker", "nunique"))
    .sort_values("score", ascending=False)
)
fig_sector = px.treemap(
    sector_summary,
    path=["sector"],
    values="names",
    color="score",
    color_continuous_scale="Blues",
    height=520,
)
show_figure(fig_sector, "Watchlist Sector Distribution")

corr_matrix = (
    build_correlation_matrix(portfolio_df["ticker"].tolist(), period="6mo")
    if not portfolio_df.empty
    else pd.DataFrame()
)
if not corr_matrix.empty:
    fig_corr = go.Figure(
        data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale="RdBu_r",
            zmid=0,
            zmin=-1,
            zmax=1,
        )
    )
    show_figure(fig_corr, "Portfolio Correlation Matrix")


In [10]:
display(Markdown("## Run Summary"))

summary_lines = [
    f"- **Universe screened:** {len(tickers)} tickers",
    f"- **Watchlist size:** {len(watchlist_df)} names",
    f"- **Portfolio size:** {len(portfolio_df)} positions",
    f"- **Sectors represented:** {watchlist_df['sector'].nunique()}",
]
if not corr_matrix.empty:
    avg_corr = corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].mean()
    summary_lines.append(f"- **Average portfolio correlation:** {avg_corr:.2f}")

display(Markdown("\n".join(summary_lines)))

display(Markdown("### Conviction Handoff"))
conviction_list = (
    portfolio_df["ticker"].tolist()
    if not portfolio_df.empty
    else watchlist_df.head(8)["ticker"].tolist()
)
display(
    Markdown(
        f"Pass these into `leaps_trade_readiness.ipynb` for deep-dive analysis:\n\n"
        f"```\nCONVICTION_TICKERS={','.join(conviction_list)}\n```"
    )
)


## Run Summary

- **Universe screened:** 45 tickers
- **Watchlist size:** 45 names
- **Portfolio size:** 10 positions
- **Sectors represented:** 7
- **Average portfolio correlation:** 0.12

### Conviction Handoff

Pass these into `leaps_trade_readiness.ipynb` for deep-dive analysis:

```
CONVICTION_TICKERS=MU,CMCSA,VZ,F,GOOGL,BAC,NU,PFE,AAL,APLD
```